In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

cleaning + preprocessing data 

In [23]:
stocks_df = pd.read_csv('stocks.csv')

/var/folders/v6/lhv05hmx5v94886x_jxd39s00000gn/T/ipykernel_20597/1353134829.py:1: DtypeWarning: Columns (9,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  stocks_df = pd.read_csv('stocks.csv')


In [24]:
stocks_df = stocks_df.drop(columns=['PERMCO', 'COMNAM', 'HEXCD', 'ISSUNO', 'TRDSTAT', 'SECSTAT', 'DISTCD'], axis=1)

In [25]:
missing_percentage = stocks_df.isnull().sum() * 100 / len(stocks_df)
columns_to_drop = missing_percentage[missing_percentage > 90].keys()
stocks_df = stocks_df.drop(columns=columns_to_drop, axis=1)

In [26]:
to_encode = stocks_df.select_dtypes(include=['object']).columns
not_encode = stocks_df.select_dtypes(exclude=['object']).columns

In [27]:
stocks_df = stocks_df.sort_values(by='date')

In [28]:
X = stocks_df[['PERMNO', 'date', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP', 'TICKER',
       'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'HSICCD', 'CUSIP', 'HSICMG',
       'HSICIG', 'BIDLO', 'ASKHI', 'VOL', 'RET', 'BID', 'ASK', 'SHROUT',
       'CFACPR', 'CFACSHR', 'OPENPRC', 'NUMTRD', 'RETX', 'vwretd', 'vwretx',
       'ewretd', 'ewretx', 'sprtrn']]

y = stocks_df[['PRC']]

In [32]:
index = int(len(stocks_df) * 0.8)
X_train = X.iloc[:index]
y_train = y.iloc[:index]
X_test = X.iloc[index:]
y_test = y.iloc[index:]

CNN-LSTM hybrid model for stock price prediction